In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('all stocks50 days.xlsx', index_col=0)

In [ ]:
df_copy = df.copy()
df_copy = df_copy.drop(columns=['Capital Gains'])
df_copy = df_copy.drop(columns=['Adj Close'])

In [ ]:
indexes = df_copy.Symbol.value_counts().loc[lambda x: x>=50].index  #get indexes that have 50 or more entries
df_g50 = df_copy[df_copy['Symbol'].isin(indexes)] #put only those indexes into df

In [ ]:
df_g50['HL day mean'] = df_g50.loc[:, 'Low'] + (df_g50.loc[:, 'High'] - df_g50.loc[:, 'Low'])/2   #high low day mean
df_g50['OC day mean'] = df_g50[['Open', 'Close']].min(axis=1) + abs(df_g50['Open'] - df_g50['Close'])/2  #open close day mean

#display(df_g50.head())

In [ ]:
#runs slow - cookie to the person who can speed it up

#find the median for the first 20 days 
df_g50['Symbol HL median'] = 0
df_g50['Symbol OC median'] = 0
unique_symbols = df_g50['Symbol'].unique()

#for index in range(5):   #for testing - leave in place
 #   symbol = unique_symbols[index]

for symbol in unique_symbols:
    subset = df_g50[df_g50['Symbol'] == symbol].iloc[0:20, :]
    symbol_hl_median = subset['HL day mean'].median()
    symbol_oc_median = subset['OC day mean'].median()
    subset = df_g50[df_g50['Symbol'] == symbol]
    df_g50.loc[subset.index, 'Symbol HL median'] = symbol_hl_median
    df_g50.loc[subset.index, 'Symbol OC median'] = symbol_oc_median


In [ ]:
df_b30 = df_g50.groupby('Symbol').apply(lambda group: group.iloc[20:21, :])

In [ ]:
#compute the percent of the median the stock is closing
df_b30['OC percent'] = df_b30['Close'] / df_b30['Symbol OC median']
df_b30['HL percent'] = df_b30['Close'] / df_b30['Symbol HL median']


In [ ]:
#find stocks that are 95% of the median to 100% of the median
stocks_to_buy = df_b30[(df_b30['OC percent'] <= 1) & (df_b30['OC percent'] > .95) & (df_b30['Volume'] > 0)]

In [ ]:
display(stocks_to_buy)